In [1]:
# import libraries
%pip install selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import time
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [2]:
# add 'chromedriver' executable to PATH
import os
os.environ['PATH']+=";C:\Program Files (x86)\Google\Chrome\Application"

In [34]:
# navigate to the website
start_url="https://tematicos.plataformadetransparencia.org.mx/web/guest/informacionrelevante/"  # specify the target webpage

chrome_options = webdriver.chrome.options.Options()
chrome_options.add_argument('--headless')    
driver = webdriver.Chrome(options=chrome_options)

#driver=webdriver.Chrome()
driver.get(start_url)
time.sleep(5)

try:
    # click "DIRECTORIO"
    driver.find_element_by_xpath("//*[@id='__flex-container-menu']/div[1]/a/div[1]/img").click()
    time.sleep(5)   
except Exception as e:
    print(e)

In [35]:
# initialize the search
try:
    # click "Búsqueda avanzada por campo"
    driver.find_element_by_id('filtrosAvanzadosDirectorioId').click()
    time.sleep(8)
    # select "Federation" at Estado o Federación"
    Select(driver.find_element_by_xpath("//*[@id='estadoFederacion']")).select_by_value("33") 
    time.sleep(8)
    # click "BUSCAR"
    driver.find_element_by_xpath("//*[@id='submitBusquedaAvanzada']").click()
    time.sleep(8)
except Exception as e:
    print(e)  

In [36]:
# initialize the search
try:
    # number of results of each page (here:200)
    Select(driver.find_element_by_xpath("//*[@id='selectRegistros']")).select_by_value("200")
    time.sleep(3)
    # select records from 2018/2019/2020
    driver.find_element_by_id('a_2019').click() 
    time.sleep(3)
except Exception as e:
    print(e)

**NOTE:** The contents on each page are loaded dynamically and spontaneously, which means that if a same page (e.g., page 2) is visited at different time, the website might display different results. Therefore, to avoid the potential problem of duplicates and/or missing records, the entire scraping is manualy divided into several rounds given that selenium-based web crawlers might break down in the middle if the target element is not found/has not been loaded. For each round of scraping, the starting page as well as the ending page are manually specified.

In [56]:
# specify the last page (lastPage > 7, otherwise the codes should be modified accordingly)
lastPage = 975  
# specify the starting page for the current round of scraping
stPage = 844
# specify the ending page for the current round of scraping
edPage = 975    

currentPage = stPage
toLastPage = lastPage-currentPage
numPage = edPage-stPage + 1

results = []
for num in range(numPage):
    # collect the institions of the employees
    inst_1 = driver.find_elements_by_xpath("//tr[@class='cursor-pointer odd parent']/td[2]")
    inst_2 = driver.find_elements_by_xpath("//tr[@class='cursor-pointer even parent']/td[2]")
    # collect the names of the employees
    name_1 = driver.find_elements_by_xpath("//tr[@class='cursor-pointer odd parent']/td[3]")
    name_2 = driver.find_elements_by_xpath("//tr[@class='cursor-pointer even parent']/td[3]")
    time.sleep(10) 
    
    inst_total = inst_1 + inst_2
    name_total = name_1 + name_2
    for inst,name in zip(inst_total, name_total):
        results.append((inst.text,name.text))
    
    # calculate the difference between the number of records should be collected and the number of 
    #  records actually collected to get an impression if the crawler works well
    print([currentPage, 200 * (num + 1)-len(results)])  # 200 is the number of results on each page
        
    # go to the next page
    #  reference to the page button visiting the next page (the target element) is dependent on the number of current 
    #  page or the distance to the last page, thus an if function is introduced to deal with different circumstances
    num += 1
    currentPage += 1
    toLastPage -= 1
    if currentPage in [2,3,4,5]:
        driver.find_element_by_xpath('//*[@id="gridDirectorioFormulario_paginate"]/span/a[{}]'.format(currentPage)).click() 
        time.sleep(8)
    elif toLastPage == 1:
        driver.find_element_by_xpath("//*[@id='gridDirectorioFormulario_paginate']/span/a[5]").click() 
        time.sleep(8)
    elif toLastPage == 0 or toLastPage == -1:
        driver.find_element_by_xpath("//*[@id='gridDirectorioFormulario_paginate']/span/a[6]").click() 
        time.sleep(8)
    else:
        driver.find_element_by_xpath("//*[@id='gridDirectorioFormulario_paginate']/span/a[4]").click()
        time.sleep(8)  
        
    driver.delete_all_cookies()
    for l in [inst_1,inst_2,name_1,name_2]:
        l.clear()

[844, 0]
[845, 0]
[846, 0]
[847, 0]
[848, 0]
[849, 0]
[850, 0]
[851, 0]
[852, 0]
[853, 0]
[854, 0]
[855, 0]
[856, 0]
[857, 0]
[858, 0]
[859, 0]
[860, 0]
[861, 0]
[862, 0]
[863, 0]
[864, 0]
[865, 0]
[866, 0]
[867, 0]
[868, 0]
[869, 0]
[870, 0]
[871, 0]
[872, 0]
[873, 0]
[874, 0]
[875, 0]
[876, 0]
[877, 0]
[878, 0]
[879, 0]
[880, 0]
[881, 0]
[882, 0]
[883, 0]
[884, 0]
[885, 0]
[886, 0]
[887, 0]
[888, 0]
[889, 0]
[890, 0]
[891, 0]
[892, 8]
[893, 16]


In [55]:
# save the results as dataframe
df=pd.DataFrame(results)
df.columns=['Institución','Nombre']
print(df)

# export the results as csv file
df.to_csv('list_of_bureaucrats_2018_9.csv', encoding='utf-8-sig', index=False)

                                            Institución  \
0     Coordinación General de la Comisión Mexicana d...   
1     Fideicomiso Fondo Nacional de Habitaciones Pop...   
2     Fideicomiso Fondo Nacional de Habitaciones Pop...   
3     Coordinación General de la Comisión Mexicana d...   
4     Consejo Nacional para Prevenir la Discriminaci...   
...                                                 ...   
8595   Secretaría de Comunicaciones y Transportes (SCT)   
8596   Secretaría de Comunicaciones y Transportes (SCT)   
8597   Secretaría de Comunicaciones y Transportes (SCT)   
8598   Secretaría de Comunicaciones y Transportes (SCT)   
8599   Secretaría de Comunicaciones y Transportes (SCT)   

                           Nombre  
0     Myrna Leonor López Martinez  
1         GERARDO BAUTISTA GARCIA  
2        ROBERTO CARBAJAL CARMONA  
3           Erika Sánchez Alvarez  
4               VÍCTOR GÓMEZ LEÓN  
...                           ...  
8595           MARIO MEZA SANCHEZ  
859